<a href="https://colab.research.google.com/github/ObjectMatrix/google-colab-notebook/blob/main/vicuna_13b_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, let's install the FastChat (Vicuna) repository.

In [ ]:
!git clone https://github.com/thisserand/FastChat.git
%cd FastChat
!pip install -e .

Second, we will install the GPTQ-for-LLaMA repository which allows us to use GPTQ quantised models.

In [ ]:
!mkdir repositories
%cd repositories
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!python setup_cuda.py install
%cd ../..

Okay, now we will download the quantised model weights.

In [ ]:
!python download-model.py anon8231489123/vicuna-13b-GPTQ-4bit-128g

To be able to run the web UI inside Colab we need virtual terminals. For this, we will use the screen library.

In [ ]:
!sudo apt-get install screen

Launch the controler service in the background.

In [ ]:
!screen -S controller -dm python -m fastchat.serve.controller --host "127.0.0.1"

Launch the model worker in the background.

In [ ]:
!screen -S model_worker -dm python -m fastchat.serve.model_worker --model-path anon8231489123/vicuna-13b-GPTQ-4bit-128g --model-name vicuna-gptq --wbits 4 --groupsize 128 --host "127.0.0.1" --worker-address "http://127.0.0.1:21002" --controller-address "http://127.0.0.1:21001"

Wait a little bit (~30 seconds) before you run this cell to make sure that the model worker is up and running. Using the public URL you can interact with the Vicuna model.

In [ ]:
!python -m fastchat.serve.gradio_web_server --controller-url "http://127.0.0.1:21001" --share